## We are taking prediction results of a district, taken at different point of time (different months within a year). We are going to combine all prediction results to make a final prediction for that year. 

In [ ]:
# import all required packsges
from PIL import Image
import math
from PIL import Image
from scipy import misc
from scipy import ndimage
import imageio
import pandas as pd
import unittest
import os, sys
import numpy as np

from collections import Counter

In [ ]:
# Covert monthly predictions from .tif to .png


cities = ['Gurgaon','Hyderabad','Mumbai']
for district in cities:
    main_folder=district     #we have stored monthly predections in a folder named by district name
    print(district)
    os.makedirs(main_folder+"/pngs",exist_ok=True)
    for infile in os.listdir(main_folder):
        if infile[-4:] == ".tif":                   #reading all tif files in given folder
            im = Image.open(main_folder+"/"+infile)
            im.save(main_folder+"/pngs/"+infile[:-4]+'.png')

In [ ]:
# Defining all required functions

def min(a,b):    #find minimum between a and b
    if a<b:
        return a
    return b

def is_number_exist(num,listx):   #membership test
    listx=set(listx)
    if num in listx:
        return True
    
def count(num,listx):                       #count the occurence of 'num' in 'listx'
    keys = list(Counter(listx).keys())
    values = list(Counter(listx).values())
    flag = False
    flag = is_number_exist(num,keys)
    if flag == True:
        num_count = values[keys.index(num)]
    else:
        num_count = 0
    return num_count


  
#merge the monthly predictions and overall median prediction to get a single prediction for year
# 0 denotes Background, 1 denotes greenery, 2 denotes water, 3 benotes Built-Up, 4 denotes Barren land
def merge_prediction(x,median):
    
    #find the count of each kind of pixel value for a pixel across all the given years
    background_count = count(0,x)
    grass_count = count(1,x)
    water_count = count(2,x)
    buildings_count = count(3,x)
    bareland_count = count(4,x)
    total = len(x)
    
    #apllying different rules
    if (background_count == total):
        return '0'
    elif (water_count >= 1 and water_count > 1.5 * grass_count and grass_count>0):
        return '2'
    elif (water_count >= 0.5 * total):
        return '2'
    elif (water_count >= 1 and water_count <= 1.5 * grass_count and grass_count>0):
        return '1'
    elif (water_count != 0 and grass_count == 0 ):
        return str(median)
    elif ( grass_count >=1 ):
        return '1'
    elif((bareland_count > buildings_count) and grass_count==0 and water_count==0 ):
        return '4'
    else:
        return '3'        

In [ ]:

# Read all the different predictions of a district.

############### Change district, year, month and resolution accordingly  ################# 
years=['2016','2017','2018','2019']

for district in cities:
    print(district)
    main_folder=district     #we have stored monthly predections in a folder named by district name
    os.makedirs(main_folder+"/final",exist_ok=True)
    for year in years:
        print(year)
        Green_pixs=[]
        for infile in os.listdir(main_folder+"/pngs/"):
            if 'monthly_prediciton_' in infile and year in infile and 'median' not in infile: #reading all monthly predictions
                im = Image.open(main_folder+"/pngs/"+infile)
                month=infile[-6:-4]
                Green_pixs.append((month,np.unique(im,return_counts=True)[1][1]))
        print(Green_pixs)
        Green_pixs.sort(key=lambda x:x[1],reverse=True)
        print(Green_pixs)
        months=[Green_pixs[i][0] for i in range(min(5,len(Green_pixs)))] #taking best 5 months on the basis of greenery
        print(months)
        ##########################################################################################
        
        #change the name of selected monthly predictions (best 5 months on the basis of highest greenery) here
        
        prediction1 = 'selected_monthly_prediction1.png'
        prediction2 = 'selected_monthly_prediction1.png'
        prediction3 = 'selected_monthly_prediction1.png'
        prediction4 = 'selected_monthly_prediction1.png'
        prediction5 = 'selected_monthly_prediction1.png'
        
        
        #defining median file name
        prediction_median = 'Sentinel_SR_Gurgaon_'+year+'_year_median.png'




        prediction1_path = main_folder+'/pngs/'+prediction1
        prediction2_path = main_folder+'/pngs/'+prediction2
        prediction3_path = main_folder+'/pngs/'+prediction3
        prediction4_path = main_folder+'/pngs/'+prediction4
        prediction5_path = main_folder+'/pngs/'+prediction5

        prediction_median_path = main_folder+'/pngs/'+prediction_median




        print(prediction1_path)
        print(prediction2_path)
        print(prediction3_path)
        print(prediction4_path)
        print(prediction5_path)
        print(prediction_median_path)

# Read all the required predictions

        prediction1 = imageio.imread(prediction1_path)
        prediction2 = imageio.imread(prediction2_path)
        prediction3 = imageio.imread(prediction3_path)
        prediction4 = imageio.imread(prediction4_path)
        prediction5 = imageio.imread(prediction5_path)

        prediction_median = imageio.imread(prediction_median_path)


        print("prediction1 ",np.unique(prediction1,return_counts=True))
        print("prediction2 ",np.unique(prediction2,return_counts=True))
        print("prediction3 ",np.unique(prediction3,return_counts=True))
        print("prediction4 ",np.unique(prediction4,return_counts=True))
        print("prediction5 ",np.unique(prediction5,return_counts=True))

        print("prediction_median ",np.unique(prediction_median,return_counts=True))

        image_dimension = prediction1.shape
        print(image_dimension)

        final_prediction = np.arange(image_dimension[0] * image_dimension[1]).reshape(image_dimension)
        print(final_prediction)


        for i in range(image_dimension[0]):
            for j in range(image_dimension[1]):
                x=[ prediction1[i][j], prediction2[i][j], prediction3[i][j], prediction4[i][j], prediction5[i][j]]
                final_prediction[i,j] = merge_prediction(x, prediction_median[i][j] )

        print("final_prediction ",np.unique(final_prediction,return_counts=True))

        imageio.imwrite(main_folder+'/final/'+district +'_prediction_'+year+'.png', final_prediction[:,:])


In [ ]:
#make final predictions colorful

from PIL import Image

district='Chennai'
year='2016'
image= '_new/' +  'final_prediction_results/'+resolution+district+'/'+district +'_prediction_'+year+'.png'


img = Image.open(image)
img = img.convert("RGBA")
pixdata = img.load()
print(img.getcolors())
for y in range(img.size[1]):
    for x in range(img.size[0]):
        if pixdata[x, y] == (0, 0, 0, 255): #background
            pixdata[x, y] = (0,0,0, 0)
        elif pixdata[x, y] == (64, 64, 64, 255):      #green
            pixdata[x, y] = (34,139,34, 255)
        elif pixdata[x, y] == (127, 127, 127, 255):    #water
            pixdata[x, y] = (2, 4, 251, 255)
        elif pixdata[x, y] == (191, 191, 191, 255):   #buildingd 
            pixdata[x, y] = (255, 255, 102, 255)
        elif pixdata[x, y] == (255, 255, 255, 255):   #bareland
            pixdata[x, y] = (255, 80, 80, 255)
img.save('_new/final_prediction_results/'+resolution+district+'/'+district+'_prediction_'+year+'_colored_median2.png')